In [1]:
!unzip "/content/drive/My Drive/ADM/dataset3.zip" -d "/content/sample_data"

Streaming output truncated to the last 5000 lines.
  inflating: /content/sample_data/dataset3/validation_images_model/6542/f90e348da779a87e.jpg  
  inflating: /content/sample_data/dataset3/validation_images_model/6542/fedb11dfe8669d4f.jpg  
  inflating: /content/sample_data/dataset3/validation_images_model/6542/ff564805490e845e.jpg  
   creating: /content/sample_data/dataset3/validation_images_model/6543/
  inflating: /content/sample_data/dataset3/validation_images_model/6543/6f7a69a5bc6d40ae.jpg  
   creating: /content/sample_data/dataset3/validation_images_model/6544/
  inflating: /content/sample_data/dataset3/validation_images_model/6544/09d33fbbd531dd97.jpg  
  inflating: /content/sample_data/dataset3/validation_images_model/6544/2201eff9af5c5173.jpg  
  inflating: /content/sample_data/dataset3/validation_images_model/6544/4d1a47e26bded942.jpg  
  inflating: /content/sample_data/dataset3/validation_images_model/6544/5696ea3defda98cf.jpg  
  inflating: /content/sample_data/dataset3/

In [2]:
!unzip "/content/drive/My Drive/ADM/test_set.zip" -d "/content/sample_data"

Streaming output truncated to the last 5000 lines.
  inflating: /content/sample_data/test_set/88da7dde21d83fbe.jpg  
  inflating: /content/sample_data/test_set/88db4617e35835b8.jpg  
  inflating: /content/sample_data/test_set/88df9028b19b663a.jpg  
  inflating: /content/sample_data/test_set/88e8c93d21a53eee.jpg  
  inflating: /content/sample_data/test_set/88ec7e4b1e012c36.jpg  
  inflating: /content/sample_data/test_set/8900c71eeecd69cf.jpg  
  inflating: /content/sample_data/test_set/8905ef3ec4c276e4.jpg  
  inflating: /content/sample_data/test_set/8906c9dbe205c92b.jpg  
  inflating: /content/sample_data/test_set/890fea9fe52e3082.jpg  
  inflating: /content/sample_data/test_set/89115a38c06a5e64.jpg  
  inflating: /content/sample_data/test_set/8911a1859e0c69bc.jpg  
  inflating: /content/sample_data/test_set/8914e2ebc0dce040.jpg  
  inflating: /content/sample_data/test_set/891914d6c0025fac.jpg  
  inflating: /content/sample_data/test_set/8919c40f0eb7239d.jpg  
  inflating: /content/sam

In [3]:
cd "/content/sample_data/dataset3"


/content/sample_data/dataset3


In [4]:
import os

In [11]:
from absl import logging

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import BytesIO

import tensorflow as tf

import tensorflow_hub as hub
from six.moves.urllib.request import urlopen

In [6]:
delf = hub.load('https://tfhub.dev/google/delf/1').signatures['default']

In [7]:
def run_delf(image):
  try:
    np_image = np.array(image)
    float_image = tf.image.convert_image_dtype(np_image, tf.float32)

    return delf(
      image=float_image,
      score_threshold=tf.constant(100.0),
      image_scales=tf.constant([0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0]),
      max_feature_num=tf.constant(1000))
  except:
    return 0

In [8]:
def delf_work(path1, path2):
  image1 = Image.open(path1)
  image1 = ImageOps.fit(image1, (96, 96), Image.ANTIALIAS)
  image2 = Image.open(path2)
  image2 = ImageOps.fit(image2, (96, 96), Image.ANTIALIAS)
  result1 = run_delf(image1)
  result2 = run_delf(image2) 

  if (type(result1) is int or type(result2) is int) and (result1==0 or result2 == 0):
    return 0 

  distance_threshold = 0.8

  # Read features.
  num_features_1 = result1['locations'].shape[0]
  #print("Loaded image 1's %d features" % num_features_1)
  
  num_features_2 = result2['locations'].shape[0]
  #print("Loaded image 2's %d features" % num_features_2)

  # Find nearest-neighbor matches using a KD tree.
  d1_tree = cKDTree(result1['descriptors'])
  _, indices = d1_tree.query(
      result2['descriptors'],
      distance_upper_bound=distance_threshold)

  # Select feature locations for putative matches.
  locations_2_to_use = np.array([
      result2['locations'][i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])
  locations_1_to_use = np.array([
      result1['locations'][indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])

  # Perform geometric verification using RANSAC.
  try:
    _, inliers = ransac(
        (locations_1_to_use, locations_2_to_use),
        AffineTransform,
        min_samples=3,
        residual_threshold=20,
        max_trials=1000)
  except:
    inliers = [0]

  if inliers is None:
    print('Found 0 inliers')
    return 0
  print('Found %d inliers' % sum(inliers))
  return sum(inliers)


In [9]:
import pandas as pd

In [10]:
def local_feature(path):
  x = 0
  v=[]
  result = pd.read_csv(path)
  for i in result['id']:
    a = result.loc[result['id'] == i, "landmarks"]
    c = result.loc[result['id'] == i, "confidence"]
    a = a.values
    c = c.values
    #print(a,c)
    if c<0.4:
      p2 = '/content/sample_data/dataset3/train_images_model/'+str(a[0])
      r = os.listdir(p2)

      if(len(r)>2):
        s=0
        for j in range(2):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
      else:
        s=0
        for j in range(len(r)):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s

    
      if avg < 3:
        result.loc[result.id == i, "confidence"] = 0
      elif avg >= 10:
        result.loc[result.id == i, "confidence"] = 0.6
    elif c>0.4 and c<0.6:
      p2 = '/content/sample_data/dataset3/train_images_model/'+str(a[0])
      r = os.listdir(p2)

      if(len(r)>2):
        s=0
        for j in range(2):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
      else:
        s=0
        for j in range(len(r)):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
    elif c==1:
      p2 = '/content/sample_data/dataset3/train_images_model/'+str(a[0])
      r = os.listdir(p2)

      if(len(r)>5):
        s=0
        for j in range(5):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
      else:
        s=0
        for j in range(len(r)):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
    
      if avg < 3:
        result.loc[result.id == i, "confidence"] = 0
    #v.append([x,avg,c])
    print(x)
    x+=1


    
  result.to_csv('delf_test_best.csv')
  #return v
  #print(GAP_metric_2(result))

In [31]:
local_feature("/content/resultvgg16.csv")

Streaming output truncated to the last 5000 lines.
Found 0 inliers
11016
Found 0 inliers
Found 0 inliers
11017
11018
11019
Found 4 inliers
Found 0 inliers
11020
Found 5 inliers
Found 11 inliers
11021
11022
Found 0 inliers
Found 0 inliers
11023
11024
Found 0 inliers
Found 0 inliers
11025
Found 0 inliers
Found 0 inliers
11026
Found 0 inliers
Found 0 inliers
11027
Found 0 inliers
Found 0 inliers
11028
11029
Found 0 inliers
Found 0 inliers
11030
Found 0 inliers
Found 0 inliers
11031
11032
Found 0 inliers
Found 6 inliers
11033
Found 14 inliers
Found 27 inliers
11034
11035
Found 0 inliers
Found 0 inliers
11036
Found 0 inliers
Found 0 inliers
11037
Found 0 inliers
Found 0 inliers
11038
11039
Found 5 inliers
Found 6 inliers
11040
11041
Found 4 inliers
Found 6 inliers
11042
11043
Found 0 inliers
11044
Found 0 inliers
Found 0 inliers
11045
Found 3 inliers
Found 0 inliers
11046
Found 0 inliers
Found 0 inliers
11047
11048
11049
Found 0 inliers
Found 0 inliers
11050
Found 0 inliers
Found 0 inliers


In [46]:
# This call calulates the Global Average Precision Metric 
# 
import pandas as pd
def GAP_metric(path):
    result=pd.read_csv(path)
    test=pd.read_csv("/content/sample_data/dataset3/test_sample3.csv")
    result=result.sort_values(by='confidence', ascending=False)
    M=0
    true=0
    total=0
    s=0
    relevance=0
    for i in result['id']:
        total+=1
        a=result.loc[result['id'] == i, "landmarks"]
        b=a.values
        #if i in test['id']:
        if i in test.id.values:
          M+=1
          c=test.loc[test['id'] == i, "landmark_id"]
          d=c.values
          if b[0]==d[0]:
              true+=1
              relevance=1
        s+=(true/total)*relevance
        relevance=0
    print(M/,s/2,total,true)
    print(s/M)
    return s/M

In [34]:
GAP_metric("/content/sample_data/dataset3/delf_test_best.csv")

5305 2161.695945001019 13067 4100
0.4074827417532552


0.4074827417532552